In [20]:
import matplotlib.image as mpimg
import cv2
import os
import numpy as np
import pandas as pd
import collections
from tqdm import tqdm
from matplotlib import pyplot as plt
from random import *
#imports for image reading
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
#imports for label encoding for classes
from sklearn import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,Flatten,GlobalMaxPooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2

In [4]:
train_dogs = pd.read_csv('labels.csv')

In [5]:
train_folder = 'train/'

In [6]:
new_data = pd.DataFrame()
for i in train_dogs['breed'].unique():
    length = len(train_dogs[train_dogs['breed'] == i])
    if length >= 90:
        new_data = new_data.append(train_dogs[train_dogs['breed'] == i])
new_data

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
201,04b5834c3a969c6308ce157c547bb313,basenji
287,06faa041b335551e3ccf3c239d006425,basenji
301,073f0821a0842917e6e97ab322bd63f2,basenji
397,0990fe6be15d9d556eac8712db3c8094,basenji
...,...,...
9669,f375e6363bc21dcd3cb65637c7855e9c,silky_terrier
9708,f42afcc1fabac064a07c6a5b7ae2766e,silky_terrier
9904,f8d48f89aaa55962d4beb853a128eac7,silky_terrier
9961,fa26802c7a0ff2fc7dcabb9999a6c4b3,silky_terrier


In [7]:
train_dogs = new_data
train_dogs

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
201,04b5834c3a969c6308ce157c547bb313,basenji
287,06faa041b335551e3ccf3c239d006425,basenji
301,073f0821a0842917e6e97ab322bd63f2,basenji
397,0990fe6be15d9d556eac8712db3c8094,basenji
...,...,...
9669,f375e6363bc21dcd3cb65637c7855e9c,silky_terrier
9708,f42afcc1fabac064a07c6a5b7ae2766e,silky_terrier
9904,f8d48f89aaa55962d4beb853a128eac7,silky_terrier
9961,fa26802c7a0ff2fc7dcabb9999a6c4b3,silky_terrier


<h2> Preprocess labels.csv to add the id + .png file to be used later for loading

In [8]:
train_dogs['img_file'] = train_dogs['id'].apply(lambda x: x + ".jpg")
train_dogs

,id,breed,img_file
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,003df8b8a8b05244b1d920bb6cf451f9.jpg
201,04b5834c3a969c6308ce157c547bb313,basenji,04b5834c3a969c6308ce157c547bb313.jpg
287,06faa041b335551e3ccf3c239d006425,basenji,06faa041b335551e3ccf3c239d006425.jpg
301,073f0821a0842917e6e97ab322bd63f2,basenji,073f0821a0842917e6e97ab322bd63f2.jpg
397,0990fe6be15d9d556eac8712db3c8094,basenji,0990fe6be15d9d556eac8712db3c8094.jpg
...,...,...,...
9669,f375e6363bc21dcd3cb65637c7855e9c,silky_terrier,f375e6363bc21dcd3cb65637c7855e9c.jpg
9708,f42afcc1fabac064a07c6a5b7ae2766e,silky_terrier,f42afcc1fabac064a07c6a5b7ae2766e.jpg
9904,f8d48f89aaa55962d4beb853a128eac7,silky_terrier,f8d48f89aaa55962d4beb853a128eac7.jpg
9961,fa26802c7a0ff2fc7dcabb9999a6c4b3,silky_terrier,fa26802c7a0ff2fc7dcabb9999a6c4b3.jpg


<h2>Initialize variables

In [9]:
num_breeds = 35
im_size = 299
batch_size = 64
encoder = preprocessing.LabelEncoder()

<h2>Using the img files in labels.csv load the image and preprocess using built in preprocess_input to match the requirements of imagenet 

In [11]:
#create a numpy array of the shape
#(number of dataset records, image size , image size, 3 for rgb channel ayer)
#this will be input for model
train_x = np.zeros((len(new_data), im_size, im_size, 3), dtype='float32')
train_file = 'train/' 
#iterate over img_file column of our dataset
for i, img_id in enumerate(new_data['img_file']):
  #read the image file and convert into numeric format
  #resize all images to one dimension i.e. 224x224
  #we will get array with the shape of
  # (224,224,3) where 3 is the RGB channels layers
    img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  #scale array into the range of -1 to 1.
  #preprocess the array and expand its dimension on the axis 0 
    img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  #update the train_x variable with new element
    train_x[i] = img_array

<h3>label encode classes 

In [12]:
#This will be the target for the model.
#convert breed names into numerical format
train_y = encoder.fit_transform(new_data["breed"].values)

In [13]:
train_y

array([ 3,  3,  3, ..., 32, 32, 32])

In [14]:
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42,stratify=train_y)
print("training data: (",x_train.shape, ",",y_train.shape,")")
print("test data: (",x_test.shape, ",",y_test.shape,")")

training data: ( (2869, 299, 299, 3) , (2869,) )
test data: ( (718, 299, 299, 3) , (718,) )


In [15]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.4,random_state=42,stratify=y_train)
print("training data: (",x_train.shape, ",",y_train.shape,")")
print("test data: (",x_val.shape, ",",y_val.shape,")")

training data: ( (1721, 299, 299, 3) , (1721,) )
test data: ( (1148, 299, 299, 3) , (1148,) )


In [16]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
 
#generate images for training sets 
train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=batch_size)
 
#same process for Testing sets also by declaring the instance
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_val, 
                                     y_val, 
                                     batch_size=batch_size)

In [17]:
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(299, 299, 3))

87916544/87910968 [==============================] - 37s 0us/step


In [18]:
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_breeds, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [21]:
model.compile(Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [22]:
epochs=10
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_val.shape[0] // batch_size)

Epoch 1/10
26/26 [==============================] - 121s 5s/step - loss: 3.1308 - accuracy: 0.3138 - val_loss: 2.4228 - val_accuracy: 0.7831
Epoch 2/10
26/26 [==============================] - 121s 5s/step - loss: 1.9751 - accuracy: 0.7767 - val_loss: 1.3071 - val_accuracy: 0.8952
Epoch 3/10
26/26 [==============================] - 123s 5s/step - loss: 1.1048 - accuracy: 0.8588 - val_loss: 0.6694 - val_accuracy: 0.9329
Epoch 4/10
26/26 [==============================] - 121s 5s/step - loss: 0.6706 - accuracy: 0.8932 - val_loss: 0.4207 - val_accuracy: 0.9421
Epoch 5/10
26/26 [==============================] - 122s 5s/step - loss: 0.5158 - accuracy: 0.8944 - val_loss: 0.3280 - val_accuracy: 0.9412
Epoch 6/10
26/26 [==============================] - 121s 5s/step - loss: 0.4232 - accuracy: 0.9040 - val_loss: 0.2662 - val_accuracy: 0.9467
Epoch 7/10
26/26 [==============================] - 122s 5s/step - loss: 0.3603 - accuracy: 0.9161 - val_loss: 0.2317 - val_accuracy: 0.9586
Epoch 8/10
26

In [23]:
predictions=model.predict(x_test)
probs = np.argmax(predictions, axis=1)
num_correct = (probs == y_test).sum()
accuracy = num_correct/len(y_test)
print('{:.4f}'.format(accuracy.item()))

0.9666
